# Bert4ColdStart using cold_data_03.csv dataset

# Import needed Libararies

In [1]:
import pandas as pd
import numpy as np

from copy import deepcopy
import torch
import random
import time
import argparse

import torch
import random
import time
import argparse
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
import os


import torch
import json
import joblib
import pickle
import torch.utils.data as data_utils
import numpy as np
import scipy.sparse as sp
from pathlib import Path
from sklearn.metrics import log_loss, roc_auc_score
from torch.utils.data.distributed import DistributedSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


import torch.nn as nn
import torch.nn.functional as F
import math

tqdm.pandas()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set to the index of the GPU you want to use
torch.cuda.set_per_process_memory_fraction(0.8)  # Limit memory usage to 80%


import warnings

#Ignore warning
warnings.filterwarnings("ignore")

2023-07-27 06:25:41.216431: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 06:25:41.835970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Checks whether a GPU is available for use with PyTorch or not

In [2]:
print(torch.cuda.is_available())

True


# Configuration
## here we are setting all the needed setting to make any changes in the project from one place

In [3]:
args = {
    'sbr_sample_size':10000000, #the selected sample size from SBR file
    'seed': 0,
    'task_name': '',
    'task_num': 4,
    'dataset_path': '',
    'pretrain_path': '',
    'source_path': 'sbr_data_1M.csv',
    'target_path': 'cold_data_0.3.csv',
    'train_batch_size': 1024,
    'val_batch_size': 1024,
    'test_batch_size': 1024,
    'sample': 'random',
    'negsample_savefolder': './data/neg_data/',
    'negsample_size': 99,
    'max_len': 20,
    'item_min': 5,
    'save_path': './checkpoint/',
    'task': -1,
    'valid_rate': 100,
    'model_name': 'BERT_ColdstartModel',
    'epochs': 20,
    're_epochs': 20,
    'lr': 0.0005,
    'device': 'cuda',
    'is_parallel': False,
    'local_rank': None,
    'num_gpu': 1,
    'weight_decay': 0.0,
    'decay_step': 5,
    'gamma': 0.5,
    'num_users': 1,
    'num_items': 1,
    'num_embedding': 1,
    'num_labels': 1,
    'k': 20,
    'metric_ks': [5, 20],
    'best_metric': 'NDCG@10',
    'hidden_size': 128,
    'block_num': 2,
    'num_groups': 4,
    'num_heads': 4,
    'dropout': 0.3,
    'bert_mask_prob': 0.3,
    'factor_num': 128,
    'embedding_size': 128,
    'dilations': [1, 4],
    'kernel_size': 3,
    'is_mp': False,
    'pad_token': 0,
    'temp': 7,
    'l2_emb': 0.0,
    'mtl_task_num': 1,
    'test_method': 'ufo',
    'val_method': 'ufo',
    'test_size': 0.1,
    'val_size': 0.1111,
    'cand_num': 100,
    'sample_method': 'high-pop',
    'sample_ratio': 0.3,
    'num_ng': 4,
    'loss_type': 'BPR',
    'init_method': 'default',
    'optimizer': 'default',
    'early_stop': True,
    'reg_1': 0.0,
    'reg_2': 0.0,
    'context_window': 2,
    'rho': 0.5,
    'node_dropout': 0.1,
    'mess_dropout': 0.1,
    'hidden_size_list': [128, 128],
    'latent_dim': 128,
    'anneal_cap': 0.2,
    'total_anneal_steps': 1000,
    'kd': False,
    'alpha': 0.4,
    'add_num_times': 2,
    'is_pretrain': 1,
    'user_profile': 'gender',
    'prun_rate': 0,
    'll_max_itemnum': 0,
    'lifelong_eval': True,
    'task1_out': 0,
    'task2_out': 0,
    'task3_out': 0,
    'task4_out': 0,
    'eval': True,
    'ch':True,
}

# Data Preparation

In [4]:
#Select sample from the SBR file sbr_data_1M.csv as it's to large to be computed

def generate_sbr_sample():
    df = pd.read_csv("sbr_data_1M.csv")
    sampled_df = df.sample(args['sbr_sample_size'])
    print('sample size : ', sampled_df.count())
    file_name = "sbr_sample_"+str(args['sbr_sample_size'])+".csv"
    args['source_path'] = file_name #update the source path with the generated sampled SBR
    sampled_df.to_csv(file_name, index=False)

In [5]:
#Item ID Resetter with Label Encoding
class item_reset_df(object):

    def __init__(self):
        print("=" * 10, "Initialize Reset DataFrame Object", "=" * 10)
        self.item_enc = LabelEncoder()

    def fit_transform(self, df):
        print("=" * 10, "Resetting item ids in DataFrame", "=" * 10)
        df['item_id'] = self.item_enc.fit_transform(df['item_id']) + 1
        return df

    def inverse_transform(self, df):
        df['item_id'] = self.item_enc.inverse_transform(df['item_id']) - 1
        return df
    
#Construct training data
def construct_data(args, item_min):
    path1 = args['target_path']
    path2 = args['source_path']
    df1 = pd.read_csv(path1, usecols=['user_id', 'item_id', 'click','gender']).head(100000)
    df1=df1[df1['gender']==2]
    df1 = df1[df1.click.isin([1])]
    df2 = pd.read_csv(path2, usecols=['user_id', 'item_id', 'click','gender'])
    df2 = df2[df2.click.isin([1])]
    user_counts = df2.groupby('user_id').size()
    user_subset = np.in1d(df2.user_id, user_counts[user_counts >= item_min].index)
    df2 = df2[user_subset].reset_index(drop=True)

    assert (df2.groupby('user_id').size() < item_min).sum() == 0
    s_item_count = len(set(df2['item_id']))
    reset_ob = cold_reset_df()
    df2, df1 = reset_ob.fit_transform(df2, df1)
    user1 = set(df1.user_id.values.tolist())
    user2 = set(df2.user_id.values.tolist())
    user = user1 & user2
    df1 = df1[df1.user_id.isin(list(user))]
    df2 = df2[df2.user_id.isin(list(user))]
    new_data1 = []
    new_data2 = []
    for u in user:
        tmp_data2 = df2[df2.user_id == u][:-3].values.tolist()
        tmp_data1 = df1[df1.user_id == u].values.tolist()
        new_data1.extend(tmp_data1)
        new_data2.extend(tmp_data2)
        
    new_data1 = pd.DataFrame(new_data1, columns=df1.columns)
    new_data2 = pd.DataFrame(new_data2, columns=df2.columns)
    user_count = len(set(new_data1.user_id.values.tolist()))
    reset_item = item_reset_df()
    new_data1 = reset_item.fit_transform(new_data1)
    t_item_count = len(set(new_data1['item_id']))
    return new_data1, new_data2, user_count, t_item_count, s_item_count

#Construct cold-start dataset
def colddataset(item_min, args, path=None):
    target_data, source_data, user_count, t_item_count, s_item_count = construct_data(args, item_min)
    print("+++user_history+++")
    user_history = source_data.groupby('user_id').item_id.apply(list).to_dict()
    target = target_data.groupby('user_id').item_id.apply(list).to_dict()

    examples = []
    for u, t_list in tqdm(target.items()):
        for t in t_list:
            e_list = [user_history[u] + [0], t]
            examples.append(e_list)
    examples = pd.DataFrame(examples, columns=['source', 'target'])
    return examples, user_count, s_item_count, t_item_count


class cold_reset_df(object):

    def __init__(self):
        print("=" * 10, "Initialize Reset DataFrame Object", "=" * 10)
        self.item_enc1 = LabelEncoder()
        self.item_enc2 = LabelEncoder()
        self.user_enc = LabelEncoder()

    def fit_transform(self, df1, df2):
        print("=" * 10, "Resetting user ids and item ids in DataFrame", "=" * 10)
        df = pd.concat([df1['user_id'], df2['user_id']])
        df = self.user_enc.fit_transform(df) + 1
        df1['item_id'] = self.item_enc1.fit_transform(df1['item_id']) + 1
        df1['user_id'] = df[:len(df1)]
        df2['item_id'] = self.item_enc2.fit_transform(df2['item_id']) + 1
        df2['user_id'] = df[len(df1):]
        return df1, df2

    def inverse_transform(self, df):
        df['item_id'] = self.item_enc.inverse_transform(df['item_id']) - 1
        df['user_id'] = self.user_enc.inverse_transform(df['user_id']) - 1
        return df

class ColdDataset(data_utils.Dataset):
    def __init__(self, x, y, max_len, mask_token):
        self.seqs = x
        self.targets = y
        self.max_len = max_len
        self.mask_token = mask_token

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, index):
        seq = self.seqs[index]
        target = self.targets[index]
        seq = seq[-self.max_len:]
        seq_len = len(seq)
        seq_mask_len = self.max_len - seq_len
        seq = [0] * seq_mask_len + seq
        return torch.LongTensor(seq), torch.LongTensor([target])

class ColdEvalDataset(data_utils.Dataset):
    def __init__(self, x, y, max_len, mask_token, num_item):
        self.seqs = x
        self.targets = y
        self.max_len = max_len
        self.mask_token = mask_token
        self.num_item = num_item + 1

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, index):
        seq = self.seqs[index]
        target = self.targets[index]
        labels = [0] * self.num_item
        labels[target] = 1
        seq = seq[-self.max_len:]
        seq_len = len(seq)
        seq_mask_len = self.max_len - seq_len
        seq = [self.mask_token] * seq_mask_len + seq
        return torch.LongTensor(seq), torch.LongTensor(labels)

# create and return a PyTorch data loader for a training dataset
def get_train_loader(dataset, args):
    if args['is_parallel']:
        dataloader = data_utils.DataLoader(dataset, batch_size=args['train_batch_size'], sampler=DistributedSampler(dataset))
    else:
        dataloader = data_utils.DataLoader(dataset, batch_size=args['train_batch_size'], shuffle=True, pin_memory=True)
    return dataloader

# create and return a PyTorch data loader for a validation dataset
def get_val_loader(dataset, args):
    if args['is_parallel']:
        dataloader = data_utils.DataLoader(dataset, batch_size=args['val_batch_size'], sampler=DistributedSampler(dataset))
    else:
        dataloader = data_utils.DataLoader(dataset, batch_size=args['val_batch_size'], shuffle=False, pin_memory=True)
    return dataloader

# create and return a PyTorch data loader for a testing dataset
def get_test_loader(dataset, args):
    if args['is_parallel']:
        dataloader = data_utils.DataLoader(dataset, batch_size=args['test_batch_size'], sampler=DistributedSampler(dataset))
    else:
        dataloader = data_utils.DataLoader(dataset, batch_size=args['test_batch_size'], shuffle=False, pin_memory=True)
    return dataloader


In [6]:
# Data Preparation and Splitting Function for Cold Start Recommendation System
def get_data(args):
    path = args['dataset_path']
    rng = random.Random(args['seed'])
    data, user_count, vocab_size, item_count = colddataset(args['item_min'], args)

    x_train, x_test, y_train, y_test = train_test_split(data.source.values.tolist(),
                                                            data.target.values.tolist(),
                                                            test_size=0.2, random_state=args['seed'])
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=args['seed'])
    # not hot and cold#####

    args['num_users'] = user_count
    args['num_items'] = item_count
    args['num_embedding'] = vocab_size

    train_dataset, valid_dataset = ColdDataset(x_train, y_train, args['max_len'], args['pad_token']), ColdEvalDataset(
        x_val, y_val, args['max_len'], args['pad_token'], args['num_items'])
    test_dataset = ColdEvalDataset(x_test, y_test, args['max_len'], args['pad_token'], args['num_items'])
    train_dataloader = get_train_loader(train_dataset, args)
    valid_dataloader = get_val_loader(valid_dataset, args)
    test_dataloader = get_test_loader(test_dataset, args)
    return train_dataloader, valid_dataloader, test_dataloader
    

# Bert4ColdStart

In [7]:
class Attention(nn.Module):
    """
    Compute 'Scaled Dot Product Attention
    """

    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) \
                 / math.sqrt(query.size(-1))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn

class GELU(nn.Module):
    """
    Paper Section 3.4, last paragraph notice that BERT used the GELU instead of RELU
    """

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = GELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))



class MultiHeadedAttention(nn.Module):
    """
    Take in model size and number of heads.
    """

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h

        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linear_layers, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x)

class PositionalEmbedding(nn.Module):

    def __init__(self, max_len, d_model):
        super().__init__()

        # Compute the positional encodings once in log space.
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size = x.size(0)
        return self.pe.weight.unsqueeze(0).repeat(batch_size, 1, 1)

class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)

class BERTEmbedding(nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos
        2. SegmentEmbedding : adding sentence segment info, (sent_A:1, sent_B:2)

        sum of all these features are output of BERTEmbedding
    """

    def __init__(self, vocab_size, embed_size, max_len, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(max_len=max_len, d_model=embed_size)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    def forward(self, sequence):
        x = self.token(sequence) + self.position(sequence)
        return self.dropout(x)

class TransformerBlock(nn.Module):
    """
    Bidirectional Encoder = Transformer (self-attention)
    Transformer = MultiHead_Attention + Feed_Forward with sublayer connection
    """

    def __init__(self, hidden, attn_heads, feed_forward_hidden, dropout):
        """
        :param hidden: hidden size of transformer
        :param attn_heads: head sizes of multi-head attention
        :param feed_forward_hidden: feed_forward_hidden, usually 4*hidden_size
        :param dropout: dropout rate
        """

        super().__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model=hidden, d_ff=feed_forward_hidden, dropout=dropout)
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return self.dropout(x)

class bottle_net(nn.Module):
    def __init__(self, hidden):
        super(bottle_net, self).__init__()
        self.hidden = hidden
        self.hidden_size = int(hidden / 4)
        self.linear1 = nn.Linear(self.hidden, self.hidden_size)
        self.linear2 = nn.Linear(self.hidden_size, self.hidden)
        self.ln = nn.LayerNorm(self.hidden, eps=1e-8)

    def forward(self, x):
        out = self.linear1(x)
        out = F.leaky_relu(out)
        out = self.linear2(out)
        out = self.ln(out)
        return out

class BERT(nn.Module):
    def __init__(self, args):
        super().__init__()
        max_len =   args['max_len']
        n_layers =  args['block_num']
        heads = args['num_heads']
        vocab_size = args['num_embedding'] + 1
        hidden = args['hidden_size'] 
        self.hidden = hidden
        self.is_mp = args['is_mp']
        dropout = args['dropout']

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=self.hidden, max_len=max_len, dropout=dropout)

        transformer_blocks = []
        for _ in range(n_layers):
            transformer_blocks.append(TransformerBlock(hidden, heads, hidden * 4, dropout))
            if self.is_mp:
                transformer_blocks.append(bottle_net(self.hidden))
        self.transformer_blocks = nn.ModuleList(transformer_blocks)

    def forward(self, x):
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x)
        # running over multiple transformer blocks
        for i, block in enumerate(self.transformer_blocks):
            mp_input = x
            if i % 2 == 0:
                x = block.forward(x, mask)
            else:
                if self.is_mp:
                    mp_out = block(mp_input)
                    x = mp_out + x
        return x

    def init_weights(self):
        pass

class BERT_ColdstartModel(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.bert = BERT(args)
        self.num_items = args['num_items']
        self.out = nn.Linear(self.bert.hidden*args['max_len'], args['num_items'] + 1)#update

    def forward(self, x):
        x = self.bert(x)
        
        return self.out(x.view(-1,self.bert.hidden*args['max_len']))#update

    def predict(self, x, item):
        x = self.bert(x)
        item_emb = self.bert.embedding.token(item)
        logits = x.matmul(item_emb.transpose(1, 2))
        print('shape logits')
#         print(logits.shape)
        logits = logits.mean(1)
#         print(logits.shape)

        return logits

In [8]:
#generic function get the configured model in args
def get_model(args, linear_feature_columns=None, dnn_feature_columns=None, history_feature_list=None):
    if args['model_name'] == 'BERT_ColdstartModel':
        return BERT_ColdstartModel(args)
    elif args['model_name'] == 'Peter4Coldstart':
        return Peter4Coldstart(args)
    
    # elif name == 'vae':
    #     return VAECF(args)
    # elif name == 'item2vec':
    #     return Item2Vec(args)
    else:
        raise ValueError('unknown model name: ' +  args['model_name'])

In [9]:
def set_seed(seed, re=True):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    if re:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
    else:
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

# Trainer

In [10]:
# perform validation on a recommendation model.
def Sequence_full_Validate(epoch, model, dataloader, writer, args, test=False):
    print("+" * 20, "Valid Epoch {}".format(epoch + 1), "+" * 20)
    model.eval()
    avg_metrics = {}
    i = 0
    with torch.no_grad():
        tqdm_dataloader = dataloader
        for data in tqdm_dataloader:
            data = [x.to(args['device']) for x in data]
            seqs, labels = data
            if test:
                scores = model.predict(seqs)
            else:
                scores = model(seqs)
#             scores = scores.mean(1) #update
#             print('score')
#             print(scores.shape)
#             print(labels.shape)
            metrics = recalls_and_ndcgs_for_ks(scores, labels, args['metric_ks'], args)
            i += 1
            for key, value in metrics.items():
                if key not in avg_metrics:
                    avg_metrics[key] = value
                else:
                    avg_metrics[key] += value
    for key, value in avg_metrics.items():
        avg_metrics[key] = value / i
    print(avg_metrics)
    for k in sorted(args['metric_ks'], reverse=True):
        writer.add_scalar('Train/NDCG@{}'.format(k), avg_metrics['NDCG@%d' % k], epoch)
    return avg_metrics


In [11]:
# Compute the recall and NDCG@ks metrics 
def recalls_and_ndcgs_for_ks(scores, labels, ks, args):
    metrics = {}

    answer_count = labels.sum(1)
    answer_count_float = answer_count.float()
    labels_float = labels.float()
#     print(scores.shape)
    rank = (-scores).argsort(dim=0)
    cut = rank
    for k in sorted(ks, reverse=True):
#        print(cut.shape)
       cut = cut[:, :k]
#        print(labels_float.shape)
       hits = torch.gather(labels_float,1,cut)
#         labels_float.gather( cut,0)
       metrics['Recall@%d' % k] = (hits.sum(1) / answer_count_float).mean().item()

       position = torch.arange(2, 2+k)
       weights = 1 / torch.log2(position.float()).to(args['device'])
#        print(weights.shape)
#        print(hits.shape) 
       dcg = (hits * weights).sum(1)
       idcg = torch.Tensor([weights[:min(n, k)].sum() for n in answer_count]).to(args['device'])
       ndcg = (dcg / idcg).mean()
       metrics['NDCG@%d' % k] = ndcg

    return metrics

In [12]:
# Train a sequence-based recommendation model for one epoc
def SequenceTrainer(epoch, model, dataloader, optimizer, writer, args): #schedular,
    print("+" * 20, "Train Epoch {}".format(epoch + 1), "+" * 20)
    model.train()
    running_loss = 0
    loss_fn = nn.CrossEntropyLoss(ignore_index=0)
    for data in dataloader:
        optimizer.zero_grad()
        data = [x.to(args['device']) for x in data]
        seqs, labels = data
#         print('data shape',labels.shape)
        logits = model(seqs) # B x T x V
#         print('pred shape',logits.shape)
        if 'cold' in args['task_name'] or ('life_long' in args['task_name'] and args['task'] != 0):
            logits = logits.mean(1)
            labels = labels.view(-1)
        else:
            logits = logits.view(-1, logits.size(-1)) # (B*T) x V
            labels = labels.view(-1)  # B*T

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().cpu().item()
    writer.add_scalar('Train/loss', running_loss / len(dataloader), epoch)
    print("Training CE Loss: {:.5f}".format(running_loss / len(dataloader)))
    return optimizer



In [13]:
def SeqTrain(epochs, model, train_loader, val_loader, writer, args):
    if args['is_pretrain'] == 0:
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                     lr=args['lr'], weight_decay=args['weight_decay'])
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

    model = model.to(args['device'])
    if args['is_parallel']:
        model = torch.nn.parallel.DistributedDataParallel(model,  find_unused_parameters=True,device_ids=[args['local_rank']], output_device=[args['local_rank']])
    best_metric = 0
    all_time = 0
    val_all_time = 0
    for epoch in range(epochs):
        since = time.time()
        optimizer = SequenceTrainer(epoch, model, train_loader, optimizer, writer, args)
        tmp = time.time() - since
        print('one epoch train:', tmp)
        all_time += tmp
        val_since = time.time()
        metrics = Sequence_full_Validate(epoch, model, val_loader, writer, args)
        val_tmp = time.time() - val_since
        print('one epoch val:', val_tmp)
        val_all_time += val_tmp
        if args['is_pretrain'] == 0 and 'acc' in args['task_name']:
            if metrics['NDCG@20'] >= 0.0193:
                break
        i = 1
        current_metric = metrics['NDCG@5']
        if best_metric <= current_metric:
            best_metric = current_metric
            best_model = deepcopy(model)
            state_dict = model.state_dict()
            if 'life' in args['task_name']:
                torch.save(state_dict, os.path.join(args['save_path'],
                                                         '{}_{}_seed{}_task_{}_best_model.pth'.format('sequence',
                                                                                                      args['model_name'],
                                                                                                      args['seed'],
                                                                                                      args['task'])))
            else:
                torch.save(state_dict, os.path.join(args['save_path'], '{}_{}_seed{}_is_pretrain_{}_best_model_lr{}_wd{}_block{}_hd{}_emb{}.pth'.format(args['task_name'], args['model_name'], args['seed'], args['is_pretrain'],
                                                                                                                              args['lr'], args['weight_decay'], args['block_num'], args['hidden_size'], args['embedding_size'])))
        else:
            i += 1
            if i == 10:
                print('early stop!')
                break
    print('train_time:', all_time)
    print('val_time:', val_all_time)
    return best_model

# Main Function

In [14]:
if args['is_parallel']:
    torch.distributed.init_process_group(backend="nccl")
    torch.cuda.set_device(args['local_rank'])
device = torch.device(args['device'])
set_seed(args['seed'])
writer = SummaryWriter()
print('=============cold_start=============')
train_loader, val_loader, test_loader = get_data(args) #, user_noclick
print('generate sample from SBR file')
generate_sbr_sample()
##########################################################

=============cold_start=============
========== Initialize Reset DataFrame Object ==========
========== Resetting user ids and item ids in DataFrame ==========
========== Initialize Reset DataFrame Object ==========
========== Resetting item ids in DataFrame ==========
+++user_history+++


100%|██████████| 838/838 [00:00<00:00, 48236.19it/s]


generate sample from SBR file
sample size :  user_id           10000000
item_id           10000000
click             10000000
follow            10000000
like              10000000
share             10000000
video_category    10000000
watching_times    10000000
gender            10000000
age               10000000
dtype: int64


In [15]:
def process_training():
    if args['is_pretrain'] == 1:
        print("pretrain")
        model = get_model(args)
        SeqTrain(args['epochs'], model, train_loader, val_loader, writer, args) #, user_noclick
        writer.close()
    elif args['is_pretrain'] == 2:
        args['is_mp'] = False
        model = get_model(args)
        SeqTrain(args['epochs'], model, train_loader, val_loader, writer, args)
        writer.close()
    else:
        print("transfer")
        best_weight = torch.load(args['pretrain_path'])
        if 'peter' in args['model_name']:
            args.is_mp = True
            best_weight.pop('final_layer.weight')
            best_weight.pop('final_layer.bias')
        if 'bert' in args.model_name:
            best_weight.pop('out.weight')
            best_weight.pop('out.bias')

        model = get_model(args)

        model_state = model.module.state_dict() if args.is_parallel else model.state_dict()
        best_weight = {k: v for k, v in best_weight.items() if k in model_state}
        model_state.update(best_weight)
        model.load_state_dict(model_state)
            # if 'peter' in args.model_name:
            #     for name, parm in model.named_parameters():
            #         if 'rez' not in name and 'mp' not in name and 'final_layer' not in name:
            #             parm.requires_grad = False
            # else:
            #     for name, parm in model.named_parameters():
            #         if 'out' not in name:
            #             parm.requires_grad = False
        SeqTrain(args.epochs, model, train_loader, val_loader, writer, args)
        writer.close()
        
    if args['eval']:
        model = get_model(args)
        best_weight = torch.load(os.path.join(args['save_path'],
                                                  '{}_{}_seed{}_is_pretrain_{}_best_model_lr{}_wd{}_block{}_hd{}_emb{}.pth'.format(args['task_name'], args['model_name'], args['seed'], args['is_pretrain'],
                                                                                                                                  args['lr'], args['weight_decay'], args['block_num'], args['hidden_size'], args['embedding_size'])))
        model.load_state_dict(best_weight)
        model = model.to(args['device'])
        metrics = Sequence_full_Validate(0, model, test_loader, writer, args)

        
process_training()

pretrain
++++++++++++++++++++ Train Epoch 1 ++++++++++++++++++++
Training CE Loss: 11.04710
one epoch train: 1.4711081981658936
++++++++++++++++++++ Valid Epoch 1 ++++++++++++++++++++
{'Recall@20': 0.002227171634634336, 'NDCG@20': tensor(0.0006, device='cuda:0'), 'Recall@5': 0.0, 'NDCG@5': tensor(0., device='cuda:0')}
one epoch val: 4.0783531665802
++++++++++++++++++++ Train Epoch 2 ++++++++++++++++++++
Training CE Loss: 8.69827
one epoch train: 1.3721246719360352
++++++++++++++++++++ Valid Epoch 2 ++++++++++++++++++++
{'Recall@20': 0.00330088275950402, 'NDCG@20': tensor(0.0011, device='cuda:0'), 'Recall@5': 0.0006510416666666666, 'NDCG@5': tensor(0.0003, device='cuda:0')}
one epoch val: 3.5827767848968506
++++++++++++++++++++ Train Epoch 3 ++++++++++++++++++++
Training CE Loss: 7.19103
one epoch train: 1.3593556880950928
++++++++++++++++++++ Valid Epoch 3 ++++++++++++++++++++
{'Recall@20': 0.0006510416666666666, 'NDCG@20': tensor(0.0002, device='cuda:0'), 'Recall@5': 0.0, 'NDCG@5': te

## Results
### we can summarize the evaluation results as:

#### Recall@20 : 0.0013020833333333333

#### NDCG@20 : 0.0005

#### Recall@5 : 0.0006510416666666666

#### NDCG@5 : 0.0003


# Hyperparameter Tuning - Trial 1

## will perofrm changes in the Hyperparameter and check if the accuracy enhanced or not

In [16]:
writer = SummaryWriter()
print('============= Hyperparameter Tuning - cold_start =============')
train_loader, val_loader, test_loader = get_data(args) #, user_noclick

print('generate sample from SBR file')
generate_sbr_sample()

# change l

args['lr']  = 0.0003

##########################################################
process_training()

============= Hyperparameter Tuning - cold_start =============
========== Initialize Reset DataFrame Object ==========
========== Resetting user ids and item ids in DataFrame ==========
========== Initialize Reset DataFrame Object ==========
========== Resetting item ids in DataFrame ==========
+++user_history+++


100%|██████████| 596/596 [00:00<00:00, 64572.76it/s]

generate sample from SBR file


sample size :  user_id           10000000
item_id           10000000
click             10000000
follow            10000000
like              10000000
share             10000000
video_category    10000000
watching_times    10000000
gender            10000000
age               10000000
dtype: int64
pretrain
++++++++++++++++++++ Train Epoch 1 ++++++++++++++++++++
Training CE Loss: 10.52133
one epoch train: 1.0111587047576904
++++++++++++++++++++ Valid Epoch 1 ++++++++++++++++++++
{'Recall@20': 0.0026041666666666665, 'NDCG@20': tensor(0.0008, device='cuda:0'), 'Recall@5': 0.0006510416666666666, 'NDCG@5': tensor(0.0003, device='cuda:0')}
one epoch val: 2.28291392326355
++++++++++++++++++++ Train Epoch 2 ++++++++++++++++++++
Training CE Loss: 9.21237
one epoch train: 0.8022093772888184
++++++++++++++++++++ Valid Epoch 2 ++++++++++++++++++++
{'Recall@20': 0.0009765625, 'NDCG@20': tensor(0.0004, device='cuda:0'), 'Recall@5': 0.0003255208333333333, 'NDCG@5': tensor(0.0002, device='cuda:0')}
one

## Results
### After tuning hyper-parameters

### we can summarize the evaluation results as:

#### Recall@20 : 0.0022786458333333335

#### NDCG@20 : 0.0006

#### Recall@5 : 0.0

#### NDCG@5 : 0.



# Hyperparameter Tuning - Trial 2

In [17]:
writer = SummaryWriter()
print('============= Hyperparameter Tuning 2 - cold_start =============')
train_loader, val_loader, test_loader = get_data(args) #, user_noclick

print('generate sample from SBR file')
generate_sbr_sample()

# change learning rate and patch size
args['train_batch_size'] = 2048

args['val_batch_size'] = 2048

args['test_batch_size'] = 2048

args['lr']  = 0.0008

##########################################################
process_training()

============= Hyperparameter Tuning 2 - cold_start =============
========== Initialize Reset DataFrame Object ==========
========== Resetting user ids and item ids in DataFrame ==========
========== Initialize Reset DataFrame Object ==========
========== Resetting item ids in DataFrame ==========
+++user_history+++


100%|██████████| 590/590 [00:00<00:00, 62173.74it/s]

generate sample from SBR file


sample size :  user_id           10000000
item_id           10000000
click             10000000
follow            10000000
like              10000000
share             10000000
video_category    10000000
watching_times    10000000
gender            10000000
age               10000000
dtype: int64
pretrain
++++++++++++++++++++ Train Epoch 1 ++++++++++++++++++++
Training CE Loss: 11.09471
one epoch train: 0.7660984992980957
++++++++++++++++++++ Valid Epoch 1 ++++++++++++++++++++
{'Recall@20': 0.0003255208333333333, 'NDCG@20': tensor(0.0001, device='cuda:0'), 'Recall@5': 0.0003255208333333333, 'NDCG@5': tensor(0.0001, device='cuda:0')}
one epoch val: 2.0686943531036377
++++++++++++++++++++ Train Epoch 2 ++++++++++++++++++++
Training CE Loss: 8.81556
one epoch train: 0.9573006629943848
++++++++++++++++++++ Valid Epoch 2 ++++++++++++++++++++
{'Recall@20': 0.0009765625, 'NDCG@20': tensor(0.0005, device='cuda:0'), 'Recall@5': 0.0003255208333333333, 'NDCG@5': tensor(0.0003, device='cuda:0')}
o

## Results
### After tuning hyper-parameters

### we can summarize the evaluation results as:


#### Recall@20 : 0.0013020833333333333

#### NDCG@20 : 0.0006

#### Recall@5 : 0.0009765625

#### NDCG@5 : 0.0005


# After tuning hyper-parameters we can say that the performance became better




# References

https://github.com/yuangh-x/2022-NIPS-Tenrec

https://tenrec0.github.io/assets/doc/NIPS2022-Tenrec.pdf